# python MySQL
### - CURD operation

In [108]:
import json
import pymysql

In [109]:
with open('mysql.json','r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [110]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

### - 테이블 생성

In [111]:
sql_user = """ 
        create table if not exists users(
        uid varchar(20) NOT NULL PRIMARY KEY,
        pwd CHAR(44),
        uname VARCHAR(20) NOT NULL,
        reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
		is_deleted INT DEFAULT 0
);
"""

In [112]:
cur = conn.cursor()
cur.execute(sql_user)

0

### -테이블 변경

In [113]:
cur.execute('alter table users add column email varchar(40);')

OperationalError: (1060, "Duplicate column name 'email'")

### - 데이터 입력

In [60]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid, uname) VALUES('admin','관리자');")

1

In [61]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 12, 17, 29), 0, None)


In [63]:
# 입력한 데이터를 즉시 flush 하려면 conn.commit() 추가
sql_insert_multi = """
    INSERT INTO users(uid, uname) 
        VALUES('eskim','김은숙'),('wjlee','이우정');
"""
cur.execute(sql_insert_multi)
conn.commit()

In [64]:
sql_select = """
    SELECT uid, uname, 
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i') AS reg_date,
    is_deleted, email FROM users;
"""
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', '관리자', '2021-06-09 12:17', 0, None)
('eskim', '김은숙', '2021-06-09 12:18', 0, None)
('wjlee', '이우정', '2021-06-09 12:18', 0, None)


In [66]:
# 파라메터 이용, placeholder
uid = 'djy'
uname = '대조영'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
cur.execute(sql, (uid, uname))
conn.commit()

In [68]:
uid = 'gdhong'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, '홍길동');"
cur.execute(sql, (uid,))
conn.commit()

In [71]:
users = (('jbpark','박재범'),('vincenzo','빈센조'))
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
# 아래 for 문은 cur.executemany(sql, users) 로 대체 가능
for user in users:
    cur.execute(sql, user)
conn.commit()

### - 데이터 삭제

In [72]:

cur = conn.cursor()
sql = "DELETE FROM users WHERE uid != 'admin';"
cur.execute(sql)
conn.commit()

In [73]:
import pandas as pd
uids = ['djy','eskim','gdhong','jbpark','vincenzo','wjlee']
unames = ['대조영','김은숙','홍길동','박재범','빈센조','이우정']
df = pd.DataFrame({'uid':uids, 'uname':unames})
df

,uid,uname
0,djy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


In [74]:
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
for i in df.index:
    cur.execute(sql, (df.uid[i], df.uname[i]))
conn.commit()

### -데이터 수정

In [86]:
email = 'admin@korea.com'
uid = 'admin'
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
cur.execute(sql, (email, uid))
conn.commit()

### -------

In [89]:
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
for i in df.index:
    uid = df.uid[i]
    email= f'{uid}@email.com'
    cur.execute(sql,(email, uid))
conn.commit()

In [93]:
sql = """
    SELECT uid, uname,email,
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i') AS reg_date
    FROM users where is_deleted=0;
"""

In [114]:
# 한 레코드 조회
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchone()
result

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '%s WHERE uid=%s' at line 1")

In [115]:
# 여러 레코드 조회
cur.fetchmany(2)

ProgrammingError: execute() first

In [116]:
# 모두 조회
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
 print(row)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '%s WHERE uid=%s' at line 1")

In [117]:
# 모두 조회한 뒤 데이터프레임으로 만들기
uids, unames, emails, regs = [],[],[],[]
for row in results:
    uids.append(row[0])
    unames.append(row[1])
    emails.append(row[2])
    regs.append(row[3])
df2 = pd.DataFrame({
    'uid':uids, 'uname':unames,
    'email':emails,'reg_date':regs
})
df2

NameError: name 'results' is not defined

### -데이터 검색

In [118]:
# 결과가 하나라도 반환하는 데이터의 형태는 튜플.
keyword = '김'

sql= """
    SELECT uid
    FROM users where is_deleted=0 and uname like %s;
"""

In [119]:
cur = conn.cursor()
cur.execute(sql, keyword+'%')
result = cur.fetchone()
result

('eskim',)

In [120]:
cur = conn.cursor()
cur.execute(sql, keyword+'%')
result = cur.fetchall()
result

(('eskim',),)

In [121]:
result[0][1]

IndexError: tuple index out of range

In [122]:
cur.close()
conn.close()